In [2]:
import pandas as pd
import numpy as np
import utils

In [ ]:
# old labels
#path = ""
path ="old_labels/"
test = pd.read_csv(path + "test_subset.csv")
train_1 = pd.read_csv(path + "train_first_subset.csv")[['restaurant_code','review_title','review_body','account_name','account_id','aspect','phrase','label','new_aspect_1','new_aspect_2']]
train_2 = pd.read_csv(path + "train_second_subset.csv")[['restaurant_code','review_title','review_body','account_name','account_id','aspect','phrase','label','new_aspect_1','new_aspect_2']]
train_3 = pd.read_csv(path + "train_third_subset.csv")[['restaurant_code','review_title','review_body','account_name','account_id','aspect','phrase','label','new_aspect_1','new_aspect_2']]
train_1["label"] = pd.to_numeric(train_1["label"], errors='coerce')
train = pd.concat([train_1,train_2,train_3],axis=0)
val = pd.read_csv(path + "val_subset.csv")[['restaurant_code','review_title','review_body','account_name','account_id','aspect','phrase','label','new_aspect_1','new_aspect_2']]

test = test.reset_index()
train=train.reset_index()
val = val.reset_index()

train.head()

In [ ]:
# new labels
path = "new_labels/"
test = pd.read_csv(path + "test_subset.csv")
test["label"] = pd.to_numeric(test["label"], errors='coerce')
train_1 = pd.read_csv(path + "train_first_subset.csv")[['restaurant_code','review_title','review_body','account_name','account_id','aspect','phrase','label','new_aspect_1','new_aspect_2']]
train_2 = pd.read_csv(path + "train_second_subset.csv")[['restaurant_code','review_title','review_body','account_name','account_id','aspect','phrase','label','new_aspect_1','new_aspect_2']]
extra_neg = pd.read_csv(path + "extra_neg.csv")[["Unnamed: 0",'restaurant_code','review_title','review_body','account_name','account_id','aspect','phrase','label','new_aspect_1','new_aspect_2']]
val = pd.read_csv(path + "val_subset.csv")[['restaurant_code','review_title','review_body','account_name','account_id','aspect','phrase','label','new_aspect_1','new_aspect_2']]

index = extra_neg.loc[extra_neg["Unnamed: 0"] == 0].index

test = pd.concat([test,extra_neg.iloc[:index[1],:]],axis=0)
train = pd.concat([train_1,train_2,extra_neg.iloc[index[1] + 1:index[2],:]],axis=0)
val = pd.concat([val,extra_neg.iloc[index[2] + 1:,:]],axis=0)

test.head()

In [ ]:
for data in [train_1,train_2,train_3,train,val,test]:
    print("Total size = {}".format(data.shape[0]))
    print("Noise = {} ".format(data.loc[(data.label == 0.0) & (data.new_aspect_1.isnull()) & (data.new_aspect_2.isnull())].shape[0]))
    print("Aspect with no adjectives = {} ".format(data.loc[(data.new_aspect_1.notnull()) & (data.new_aspect_2.isnull())].shape[0]))
    print("Aspect with adjectives = {} ".format(data.loc[(data.new_aspect_1.notnull()) & (data.new_aspect_2.notnull())].shape[0]))
    print("")

In [ ]:
test.head()

####  Data Preprocessing

1. Clean Phrase (Utils) 
2. Filter the phrase column where there is an empty string 
3. Combine the aspects into 1 & aggregate the sentiment scores by averaging then sign(>0 = 1,=0 =0 <0 = -1)


In [ ]:
# clean phrases
train["phrase_lemma"] = train.phrase.apply(lambda x: utils.clean_phrase(x, lemmatize=True, stem=False))
val["phrase_lemma"] = val.phrase.apply(lambda x: utils.clean_phrase(x, lemmatize=True, stem=False))
test["phrase_lemma"] = test.phrase.apply(lambda x: utils.clean_phrase(x, lemmatize=True, stem=False))

train["phrase_stem"] = train.phrase.apply(lambda x: utils.clean_phrase(x, lemmatize=False, stem=True))
val["phrase_stem"] = val.phrase.apply(lambda x: utils.clean_phrase(x, lemmatize=False, stem=True))
test["phrase_stem"] = test.phrase.apply(lambda x: utils.clean_phrase(x, lemmatize=False, stem=True))

train.phrase = train.phrase.apply(lambda x: utils.clean_phrase(x, lemmatize=False, stem=False))
val.phrase = val.phrase.apply(lambda x: utils.clean_phrase(x, lemmatize=False, stem=False))
test.phrase = test.phrase.apply(lambda x: utils.clean_phrase(x, lemmatize=False, stem=False))

In [ ]:
# filter out rows where there are no characters
train = train.loc[(train.phrase.str.len() > 0)]
val = val.loc[(val.phrase.str.len() > 0)]
test = test.loc[(test.phrase.str.len() > 0)]

In [ ]:
# if nan label, replace with 0
train.label = train.label.apply(lambda x: 0 if np.isnan(x) else x)
val.label = val.label.apply(lambda x: 0 if np.isnan(x) else x)
test.label = test.label.apply(lambda x: 0 if np.isnan(x) else x)

In [ ]:
train.head()

In [ ]:
def sgn(x):
    if x < 0:
        return -1
    elif x == 0:
        return 0
    else:
        return 1

In [ ]:
# filter out phrases with hashtags & those with character length <=5 then get the majority (might not make sense)
# train
trainagg = train.groupby(["restaurant_code","review_title","account_name","new_aspect_1"]).agg({"phrase": " ".join,
                                                                                                "phrase_lemma": " ".join,
                                                                                                "phrase_stem":" ".join,
                                                                                                "label": "mean"})
trainagg = trainagg.reset_index()
trainagg.label = trainagg.label.map(sgn)


# test
valagg = val.groupby(["restaurant_code","review_title","account_name","new_aspect_1"]).agg({"phrase": " ".join,
                                                                                                "phrase_lemma": " ".join,
                                                                                                "phrase_stem":" ".join,
                                                                                                "label": "mean"})
valagg = valagg.reset_index()
valagg.label = valagg.label.map(sgn)

#val 
testagg = test.groupby(["restaurant_code","review_title","account_name","new_aspect_1"]).agg({"phrase": " ".join,
                                                                                                "phrase_lemma": " ".join,
                                                                                                "phrase_stem":" ".join,
                                                                                                "label": "mean"})
testagg = testagg.reset_index()
testagg.label = testagg.label.map(sgn)


In [ ]:
trainvalagg = pd.concat([trainagg, valagg],axis=0)

In [ ]:
path = ""
trainagg.to_csv(path + "train_newpreproc.csv")
valagg.to_csv(path + "val_newpreproc.csv")
testagg.to_csv(path + "test_newpreproc.csv")

###### Text Vectorisation --> used on data2
1. BoW
2. Tf-idf
3. Fasttext - can handle oov words https://www.quora.com/How-does-fastText-output-a-vector-for-a-word-that-is-not-in-the-pre-trained-model (not used)


In [36]:
trainagg = pd.read_csv("train_newpreproc.csv")
valagg = pd.read_csv("val_newpreproc.csv")
testagg = pd.read_csv("test_newpreproc.csv")
trainvalagg = pd.concat([trainagg, valagg],axis=0)

In [37]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
import nltk
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
from nltk.stem import WordNetLemmatizer
stemmer = WordNetLemmatizer()
import re
from tqdm import tqdm

In [38]:
bow = CountVectorizer(analyzer='word',lowercase=True,ngram_range=(1, 3))
bow_train = bow.fit_transform(trainvalagg.phrase)
bow_test = bow.transform(testagg.phrase)

In [39]:
tfidf = TfidfVectorizer(analyzer='word',lowercase=True,ngram_range=(1, 3))
tfidf_train = tfidf.fit_transform(trainvalagg.phrase)
tfidf_test = tfidf.transform(testagg.phrase)

In [ ]:
"""
from gensim.models.fasttext import FastText
from gensim.models import Word2Vec

# Preprocess data for fasttext word embeddings training
train_clean = [process_text(sentence) for sentence in tqdm(trainvalagg.phrase) if sentence.strip() !='']
word_tokenizer = nltk.WordPunctTokenizer()
train_word_tokens = [word_tokenizer.tokenize(sent) for sent in tqdm(train_clean)]

# Preprocess data for fasttext word embeddings testing data
test_clean = [process_text(sentence) for sentence in tqdm(test2.phrase) if sentence.strip() !='']
word_tokenizer = nltk.WordPunctTokenizer()
test_word_tokens = [word_tokenizer.tokenize(sent) for sent in tqdm(test_clean)]

# Train Fasttext word embeddings on training data
# Defining values for parameters
embedding_size = 300
window_size = 5
min_word = 5
down_sampling = 1e-2
 

fast_Text_model = FastText(train_word_tokens,
                      size=embedding_size,
                      window=window_size,
                      min_count=min_word,
                      sample=down_sampling,
                      workers = 4,
                      sg=1,
                      iter=100)
"""

In [ ]:
"""
# Save fastText gensim model
fast_Text_model.save("ftmodel/ft_model_train")
# Load saved gensim fastText model
fast_Text_model = Word2Vec.load("ftmodel/ft_model_train") """

In [ ]:
"""
# convert sentence --> average(each word embedding)
fasttext_train = [np.array([fast_Text_model[word] for word in sentence]).mean(axis=0) for sentence in train_word_tokens]
# convert sentence --> average(each word embedding)
fasttext_test = [np.array([fast_Text_model[word] for word in sentence]).mean(axis=0) for sentence in test_word_tokens]
"""

In [ ]:
"""
fasttext_train_processed = []
for row in fasttext_train:
    try:
        fasttext_train_processed.append(list(row))
    except:
        fasttext_train_processed.append([0]*300)

fasttext_test_processed = []
for row in fasttext_test:
    try:
        fasttext_test_processed.append(list(row))
    except:
        fasttext_test_processed.append([0]*300)
"""

In [ ]:
"""
fasttext_train_processed = np.array(fasttext_train_processed).reshape(-1,300)
fasttext_test_processed = np.array(fasttext_test_processed).reshape(-1,300)
"""

##### Modelling

1. Naive Bayes/Support Vector Machines (NBSVM)/Logistic Regression

(Baseline) 1. Naive Bayes -Support Vector Machines (NBSVM)- Logistic Regression

In [40]:
# import and instantiate a Multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.svm import SVC

logreg = LogisticRegression()
nb = MultinomialNB()
svm = SVC()

all_train = [bow_train,tfidf_train]
all_test = [bow_test,tfidf_test]
all_vectorisers = ["BoW","TfIdf"]

for train,test,vectoriser in zip(all_train,all_test,all_vectorisers):
    logreg.fit(train,trainvalagg.label)
    
    print("Logistic Regression + " + vectoriser)
    print("Training Data")
    print(classification_report(trainvalagg.label, logreg.predict(train)))
    print("Test Data")
    print(classification_report(testagg.label, logreg.predict(test),digits=4)) 
    
    svm.fit(train,trainvalagg.label)
    
    print("SVM + " + vectoriser)
    print("Training Data")
    print(classification_report(trainvalagg.label, svm.predict(train)))
    print("Test Data")
    print(classification_report(testagg.label, svm.predict(test),digits=4)) 

    # Fasttest dont work w NaiveBayes
    nb.fit(train,trainvalagg.label)
    print("Naive Bayes + "+ vectoriser)
    print("Training Data")
    print(classification_report(trainvalagg.label, nb.predict(train)))
    print("Test Data")
    print(classification_report(testagg.label, nb.predict(test),digits=4))    

Logistic Regression + BoW
Training Data
              precision    recall  f1-score   support

          -1       0.99      0.96      0.98       333
           0       0.98      0.99      0.99       917
           1       0.99      0.99      0.99      1236

    accuracy                           0.99      2486
   macro avg       0.99      0.98      0.99      2486
weighted avg       0.99      0.99      0.99      2486

Test Data
              precision    recall  f1-score   support

          -1     0.7021    0.3173    0.4371       104
           0     0.5000    0.7982    0.6148       218
           1     0.8368    0.7020    0.7635       453

    accuracy                         0.6774       775
   macro avg     0.6797    0.6058    0.6051       775
weighted avg     0.7240    0.6774    0.6779       775

SVM + BoW
Training Data
              precision    recall  f1-score   support

          -1       0.97      0.66      0.78       333
           0       0.79      0.97      0.87       917
 

In [33]:
(34/383)*0.2769  + (104/383)*0.5693 + (245/383)*0.6531

0.5813514360313315

In [46]:
(104/775)*0.0917 + (218/775)*0.2963 + (453/775)*0.7483

0.5330452903225806